In [ ]:
import numpy as np
import pandas as pd
from tqdm import tqdm
from haversine import haversine

In [ ]:
nodes = pd.read_csv('data/SanFrancisco_nodes.csv', index_col='osmid')
edges = pd.read_csv('data/SanFrancisco_edges.csv')
weight = pd.read_csv('data/weight.csv')

In [ ]:
def pathEstimation(start, destination):
    Open = []
    Close = []
    dict = {'NodeID': start, 'FScore': 0, 'GScore': 0, 'HScore': 0, 'ParentNode': ''}
    Close.append(dict)

    # Olist에 넣기
    i = 0
    flag = 0
    for u, v, spd in zip(edges['u'], edges['v'], edges['maxspeed']):
        if u == start:
            flag = 1
            GScore = weight[str(i)][18]

            stCo= (float(nodes.loc[start].geometry[7:-1].split(' ')[1]), float(nodes.loc[start].geometry[7:-1].split(' ')[0]))
            deCo = (float(nodes.loc[destination].geometry[7:-1].split(' ')[1]), float(nodes.loc[destination].geometry[7:-1].split(' ')[0]))
            
            HScore = (haversine(stCo, (deCo[0], stCo[1]), unit = 'm') + haversine((deCo[0], stCo[1]), deCo, unit = 'm')) * (100 / 18)
            
            FScore = GScore + HScore

            dict = {'NodeID': v, 'FScore': FScore, 'GScore': GScore, 'HScore': HScore, 'ParentNode': u}
            Open.append(dict)
        else:
            if flag == 1:
                break
        i += 1
    while Close[-1].get('NodeID') != destination:
        Open = sorted(Open, key=lambda x:x['FScore'])
        Close.append(Open[0])
        Open.pop(0)
        # Olist에 넣기
        i = 0
        flag = 0
        for u, v, spd in zip(edges['u'], edges['v'], edges['maxspeed']):
            if u == Close[-1].get('NodeID'):
                flag = 1
                for item in Close:
                    if item.get('NodeID') == Close[-1].get('NodeID'):
                        prevGScore = item.get('GScore')
                GScore = weight[str(i)][12] + prevGScore

                stCo= (float(nodes.loc[Close[-1].get('NodeID')].geometry[7:-1].split(' ')[1]), float(nodes.loc[Close[-1].get('NodeID')].geometry[7:-1].split(' ')[0]))
                deCo = (float(nodes.loc[destination].geometry[7:-1].split(' ')[1]), float(nodes.loc[destination].geometry[7:-1].split(' ')[0]))
               
                HScore = (haversine(stCo, (deCo[0], stCo[1]), unit = 'm') + haversine((deCo[0], stCo[1]), deCo, unit = 'm')) * (100 / 18)

                FScore = GScore + HScore

                dict = {'NodeID': v, 'FScore': FScore, 'GScore': GScore, 'HScore': HScore, 'ParentNode': u}
                flag_d = 0
                for item in Open:
                    if item.get('NodeID') == dict.get('NodeID'):
                        flag_d = 1
                        item['FScore'] = dict.get('FScore')
                        item['GScore'] = dict.get('GScore')
                        item['HScore'] = dict.get('HScore')
                        item['ParentNode'] = dict.get('ParentNode')
                if flag_d == 0:
                    Open.append(dict)
            else:
                if flag == 1:
                    break
            i += 1
            
    path = []
    j = -1
    node = Close[j].get('NodeID')
    path.append(node)
    while path[-1] != start:
        node = Close[j].get('ParentNode')
        path.append(node)
        while node != Close[j].get('NodeID'):
            j -= 1
    path.reverse()
    cp = ''
    for i in range(len(path) - 1):
        for j in range(4468):
            if (edges['u'][j] == path[i]):
                if (edges['v'][j] == path[i + 1]):
                    cp += str(j) + ','
    cp = cp[:-1]
    return cp